In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

D:\APP\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\APP\anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [10]:
df = pd.read_csv(r"D:\datasets\ad-dataset\ad.data", header=None, low_memory=False)

In [21]:
df['labels'] =0

In [25]:
df['labels'][df[1558]=='ad.'] = 1

D:\APP\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
y =df['labels']

In [40]:
X=df.iloc[:,:-2]
X.replace(to_replace=' *\?', value=-1, regex=True, inplace=True)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [49]:
#网格搜索参数设置
pipeline = Pipeline([
('clf', DecisionTreeClassifier(criterion='entropy'))])
parameters = {
'clf__max_depth': list((150, 155, 160)),
'clf__min_samples_split': list((3, 4, 5)),
'clf__min_samples_leaf': list((2, 3, 4))}


In [50]:
#网格搜索
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, sc\
oring='f1')
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   17.9s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('clf', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [150, 155, 160], 'clf__min_samples_split': [3, 4, 5], 'clf__min_samples_leaf': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1', verbose=1)

In [51]:
#获取最优参数
print('最佳效果：%0.3f' % grid_search.best_score_)
print('最优参数：')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))

最佳效果：0.876
最优参数：
	clf__max_depth: 150
	clf__min_samples_leaf: 4
	clf__min_samples_split: 3


In [52]:
#预测以及分类器参数报告
predictions = grid_search.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.96      0.98      0.97       697
          1       0.87      0.75      0.80       123

avg / total       0.94      0.95      0.94       820

